In [0]:
import numpy as np
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.metrics import Accuracy

Losses

In [0]:
smooth = 1.

def dice_coef(y_true, y_pred):
  ''' Computes Dice coef 

  Args: 
    y_true(Tensor): Ground truth labels
    y_pred(Tensor): Predictions

  Returns:
    float: Dice coefficient
  '''

  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [0]:
def dice_coef_loss(y_true, y_pred):
  ''' Computes Dice loss 

  Args: 
    y_true(Tensor): Ground truth labels
    y_pred(Tensor): Predictions

  Returns:
    float: Dice loss
  '''  
  
  return -dice_coef(y_true, y_pred)

In [0]:
def create_weighted_binary_crossentropy(zero_weight, one_weight):
  ''' Returns function for weighted binary crossentropy loss 
  
  Args: 
    zero_weight(float): Weight for background class
    one_weight(float): Weight for foreground class

  Returns:
    function: weighted_binary_crossentropy
  '''

  def weighted_binary_crossentropy(y_true, y_pred):
    ''' Computes weighted binary crossentropy loss 

    Args: 
      y_true(Tensor): Ground truth labels
      y_pred(Tensor): Predictions

    Returns:
      float: Weighted binary crossentropy loss
    ''' 

    b_ce = K.binary_crossentropy(y_true, y_pred)

    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce

    return K.mean(weighted_b_ce)

  return weighted_binary_crossentropy

In [0]:
def soft_dice_loss(y_true, y_pred, epsilon=1e-6):
  ''' Computes soft Dice loss 

  Args: 
    y_true(Tensor): Ground truth labels
    y_pred(Tensor): Predictions
    epsilon(float): Small value preventing from division by zero

  Returns:
    float: Soft Dice loss
  ''' 

  axes = tuple(range(1, len(y_pred.shape)-1)) 
  numerator = 2. * K.sum(y_pred * y_true, axes)
  denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
  return -K.mean(numerator / (denominator + epsilon))

Metrics

In [0]:
def pixel_wise_accuracy(y_true, y_pred):
  ''' Computes pixel wise accuracy metric

  Args: 
    y_true(np.array): Ground truth labels
    y_pred(np.array): Predictions

  Returns:
    float: Pixel wise accuracy loss
  ''' 
  
  pixel_wise = tf.keras.metrics.Accuracy()
  pixel_wise.update_state(y_true, y_pred)
  return  pixel_wise.result().numpy()

In [0]:
def dice(y_true, y_pred):
  ''' Computes Dice coef metric 

  Args: 
    y_true(np.array): Ground truth labels
    y_pred(np.array): Predictions

  Returns:
    float: Dice coef metric
  ''' 
  
  dice = dice_coef(y_true, y_pred)
  return dice.numpy()

In [0]:
def iou(y_true, y_pred):
  ''' Computes IOU metric 

  Args: 
    y_true(np.array): Ground truth labels
    y_pred(np.array): Predictions

  Returns:
    float: IOU metric
  ''' 

  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred) 

  numerator = K.sum(y_true_f * y_pred_f)
  denominator = K.sum(y_true + y_pred - y_true * y_pred)

  return ((numerator + 1) / (denominator + 1)).numpy()

In [0]:
def auc(y_true, y_pred):
  ''' Computes AUC metric

  Args: 
    y_true(np.array): Ground truth labels
    y_pred(np.array): Predictions

  Returns:
    float: AUC metric
  ''' 
  
  m = tf.keras.metrics.AUC(num_thresholds=3)
  _ = m.update_state(y_true, y_pred)

  return m.result().numpy()